Pre Processing Data for DTADV-Set

In [171]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump, load
from joblib import dump
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix

In [172]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [173]:
directory = r'C:\Users\Meyer\Documents\Dom_Detecter\own_data\DTADSV-S\csv'
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [174]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

In [175]:
df_merged = merged_dataframe

In [176]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\own_data\DTADSV-S\mapping.xlsx', engine='openpyxl')

In [177]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.src_ip, mapping_df.target))

In [178]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['src_ip'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [179]:
flow_counts = df_merged.groupby('target')['Flow'].nunique()

In [180]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            3603642
tp_link_c200         52627
netatmo_cam          26126
netatmo_weather       1340
Name: count, dtype: int64


In [181]:
df = df_merged

In [182]:
        df.replace('N/A', np.nan, inplace=True)
        df['Start_Time'] = pd.to_datetime(df['Start_Time'])
        df['End_Time'] = pd.to_datetime(df['End_Time'])
        df['Flow_Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds()

        df['Flow_Bytes/s'] = df['Flow_Volume'] / df['Flow_Duration']
        df['Flow_Packets/s'] = df['Packet_Count'] / df['Flow_Duration']

        df.loc[df['Total_Fwd_Packets'] > 0, 'Fwd_IAT'] = df['Inter_Arrival_Time']
        df.loc[df['Total_Bwd_Packets'] > 0, 'Bwd_IAT'] = df['Inter_Arrival_Time']

        feature_df = df.groupby('Flow').agg({
            'Inter_Arrival_Time': ['mean', 'std', 'max', 'min'],
            'Total_Fwd_Packets': ['sum'],
            'Total_Bwd_Packets': ['sum'],
            'Fwd_Packet_Length_Total': ['max', 'min', 'mean', 'std'],
            'Bwd_Packet_Length_Total': ['max', 'min', 'mean', 'std'],
            'Flow_Bytes/s': ['mean'],
            'Flow_Packets/s': ['mean'],
            'packet_length': ['min', 'max', 'mean', 'std', 'var'],
            'Fwd_IAT': ['sum', 'mean', 'std', 'max', 'min'],
            'Bwd_IAT': ['sum', 'mean', 'std', 'max', 'min'],
        })

In [183]:
# Adjust column names and merge
feature_df.columns = ['_'.join(col) for col in feature_df.columns]
feature_df.reset_index(inplace=True)
df = pd.merge(df, feature_df, on='Flow', how='left')

### **DTASV-Set**

In [184]:
clf = load(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\lgbm\lgbm_flow_clf_trained_model.joblib')

In [185]:
df_new = df

In [186]:
df_new.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,...,Fwd_IAT_sum,Fwd_IAT_mean,Fwd_IAT_std,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min
3683730,1562,TCP,154.52.17.92,443.0,192.168.1.99,8094.0,1308.0,ack,35840.0,51.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683731,1563,TCP,192.168.1.99,8094.0,154.52.17.92,443.0,66.0,ack,26192.0,64.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683732,1564,TCP,192.168.1.99,8094.0,154.52.17.92,443.0,375.0,ack,26192.0,64.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683733,1565,TCP,154.52.17.92,443.0,192.168.1.99,8094.0,66.0,ack,38912.0,51.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683734,1566,TCP,192.168.1.99,8094.0,154.52.17.92,443.0,455.0,ack,26192.0,64.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0


In [187]:
# Count the number of packets for each target
target_counts = df_new['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            3603642
tp_link_c200         52627
netatmo_cam          26126
netatmo_weather       1340
Name: count, dtype: int64


In [188]:
# Drop duplicate rows based on the 'Flow' column, keeping only the first occurrence
df_new = df_new.drop_duplicates(subset=['Flow'], keep='first')

In [189]:
# Count the number of packets for each target
target_counts = df_new['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            50566
tp_link_c200        4138
netatmo_cam          398
netatmo_weather       63
Name: count, dtype: int64


In [190]:
flow_counts = df_new.groupby('target')['Flow'].nunique()

In [191]:
print(flow_counts)

target
netatmo_cam          398
netatmo_weather       63
tp_link_c200        4138
unknown            50566
Name: Flow, dtype: int64


In [192]:
# Drop duplicate rows based on the 'Flow' column, keeping only the first occurrence
df_new = df_new.drop_duplicates(subset=['Flow'], keep='first')

In [193]:
df_new = pd.get_dummies(df_new, columns=['protocol'])

In [139]:
df_new['ip_id'] = df_new['ip_id'].apply(lambda x: int(x, 16) if isinstance(x, str) else x)

In [140]:
df_new = df_new.fillna(0)

In [141]:
from joblib import dump, load

# Load reference headers
ref_headers = load(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\lgbm\lgbm_flow_header.joblib')

current_headers = df_new.columns

missing_headers = [col for col in ref_headers if col not in current_headers]

for header in missing_headers:
    df_new[header] = 0

df_new = df_new[ref_headers]

In [142]:
X_new = df_new.drop(columns=[
    'target', 'src_ip', 'src_port', 'dst_ip', 'packet_eth_src', 'flow_label',
    'traffic_class', 'Flow', 'Start_Time', 'End_Time', 'next_header', 'ip_len',
    'udp_src_port', 'dsfield_ecn', 'dsfield_dscp', 'flags_mf', 'udp_checksum_status', 'udp_dst_port', 'udp_length','flags_df','ip_hdr_len',
    "packet.sniff_timestamp", "packet_length", "tcp_flags", "app_layer_protocol", "dst_port", "ttl", "Bwd_Packet_Length_Total", "Fwd_Packet_Length_Total",
    "Total_Bwd_Packets", "Total_Fwd_Packets", "ip_id", "Bwd_IAT_min", "packet_length_var", "payload_len", "Fwd_IAT_min", 
    "protocol_TCP", "protocol_UDP", "Flow_Duration", "Fwd_IAT_std", "Flow_Packets/s", "Bwd_Packet_Length_Total_mean"
    ])

In [143]:
Y_new = df_new.target

In [144]:
X_new = X_new.astype(float)

In [145]:
Y_new = Y_new.astype(str)

In [146]:
#print(X_new.isin([np.inf, -np.inf]).any())

In [147]:
X_new.replace([np.inf, -np.inf], np.nan, inplace=True)

In [148]:
X_new.fillna(0, inplace=True)

In [149]:
from sklearn.ensemble import RandomForestClassifier

In [150]:
Y_new_pred = clf.predict(X_new)

In [151]:
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])

In [152]:
new_accuracy = accuracy_score(Y_new, Y_new_pred)
print('Accuracy on new data:', new_accuracy)

Accuracy on new data: 0.7590501223601922


In [153]:
all_labels = ['amazon_echo', 'belkin_motion', 'belkin_switch', 'netatmo_cam', 'netatmo_weather', 'pixstar_photoframe', 'samsung_cam', 'smart_things', 'tp-link_cam', 'triby_speaker', 'unknown']

In [154]:
from sklearn.metrics import confusion_matrix

In [155]:
cm = confusion_matrix(Y_new, Y_new_pred, labels=all_labels)

In [156]:
import pandas as pd
cm_df = pd.DataFrame(cm, index=all_labels, columns=all_labels)
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
                    amazon_echo  belkin_motion  belkin_switch  netatmo_cam  \
amazon_echo                   0              0              0            0   
belkin_motion                 0              0              0            0   
belkin_switch                 0              0              0            0   
netatmo_cam                   0             40              0            4   
netatmo_weather               0              2              0            0   
pixstar_photoframe            0              0              0            0   
samsung_cam                   0              0              0            0   
smart_things                  0              0              0            0   
tp-link_cam                   0              0              0            0   
triby_speaker                 0              0              0            0   
unknown                     992             12             29          199   

                    netatmo_weather  pixstar_

In [157]:
cm_df.to_csv(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\lgbm\DTADSV-Set\lgbm_flow_DTADSV-Set_cm.csv')

In [158]:
Y_new_df = pd.DataFrame(Y_new, columns=['Actual'])
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])

In [159]:
result_df = pd.concat([Y_new_df, Y_new_pred_df], axis=1)

In [160]:
Y_test_pred = clf.predict(X_new)

In [161]:
df_new['Predicted_Target'] = Y_new_pred

In [162]:
wrong_predictions = df_new[df_new['target'] != df_new['Predicted_Target']]

In [163]:
wrong_predictions.head()

,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,...,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP,Predicted_Target
0,fe80::acc:68ff:fe42:391a,0.0,fe80::1e24:cdff:fe81:ac40,0.0,0.0,0,0.0,255.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,netatmo_cam
2,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,netatmo_cam
6,192.168.1.111,50985.0,51.105.71.136,443.0,60.0,ack,515.0,128.0,60.0,0.0,...,0.000035,0.0,0.0,0.0,0.0,0.0,0.0,True,False,pixstar_photoframe
11,192.168.1.111,49825.0,52.111.255.0,443.0,60.0,ack,514.0,128.0,60.0,0.0,...,10.186142,0.0,0.0,0.0,0.0,0.0,0.0,True,False,amazon_echo
14,2a02:21b4:a6e2:fe01:1e24:cdff:fe81:ac40,5353.0,ff02::fb,5353.0,0.0,0,0.0,255.0,0.0,0.0,...,117.002206,0.0,0.0,0.0,0.0,0.0,0.0,False,True,tp-link_cam


In [164]:
df_new['is_correct'] = df_new['target'] == df_new['Predicted_Target']

In [165]:
df_new.head()

,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,...,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP,Predicted_Target,is_correct
0,fe80::acc:68ff:fe42:391a,0.0,fe80::1e24:cdff:fe81:ac40,0.0,0.0,0,0.0,255.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,netatmo_cam,False
2,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,netatmo_cam,False
3,192.168.1.111,50995.0,192.168.0.23,7680.0,66.0,syn,64240.0,128.0,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,True,False,unknown,True
5,192.168.1.99,0.0,192.168.1.111,0.0,94.0,0,0.0,255.0,94.0,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,unknown,True
6,192.168.1.111,50985.0,51.105.71.136,443.0,60.0,ack,515.0,128.0,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,True,False,pixstar_photoframe,False


In [166]:
df_new.to_csv(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\lgbm\DTADSV-Set\lgbm_flow_new_data_with_predictions_DTATSV-Set.csv', index=False)

In [167]:
# Group by the true label (i.e., the actual device type)
grouped = df_new.groupby('target')

In [168]:
accuracy_per_device = {}

In [169]:
# Calculate accuracy for each device type
for name, group in grouped:
    accuracy = accuracy_score(group['target'], group['Predicted_Target'])
    accuracy_per_device[name] = accuracy

In [170]:
print("Accuracy per device type:", accuracy_per_device)

Accuracy per device type: {'netatmo_cam': 0.010050251256281407, 'netatmo_weather': 0.9682539682539683, 'tp_link_c200': 0.0, 'unknown': 0.8268006170153859}
